# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('../output_data/cities.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = df["Humidity (%)"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(center = (0.0,0.0), zoom_level=1)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# filter to max temperature lower than 80 degrees but higher than 70
ideal = df.loc[df['Max Temp (F)']>70,:]
ideal = ideal.loc[ideal['Max Temp (F)']<80,:]

# filter wind speed less than 10 mph
ideal = ideal.loc[ideal['Wind (mph)']<10,:]

# filter to zero cloudiness
ideal = ideal.loc[ideal['Cloudiness (%)'] == 0,:]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# add ideal cities to new hotel dataframe
hotel_df = ideal
# delete unneeded and trash columns
del hotel_df['Unnamed: 0']
# add empty 'hotel name' column
hotel_df['Hotel Name'] = ""

In [8]:
# set up base URL and search parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "airport",
    "key": g_key,
}

In [9]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    location = str(row['Lat']) +', '+str(row['Lng'])

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest airport is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 24.
Missing field/result... skipping.
------------
Retrieving Results for Index 242.
Closest airport is Nioaque Airport.
------------
Retrieving Results for Index 265.
Missing field/result... skipping.
------------
Retrieving Results for Index 311.
Closest airport is Morón Air Base.
------------
Retrieving Results for Index 364.
Missing field/result... skipping.
------------
Retrieving Results for Index 395.
Closest airport is Charagua Airport.
------------
Retrieving Results for Index 411.
Missing field/result... skipping.
------------
Retrieving Results for Index 429.
Closest airport is Vavuniya Airport.
------------
Retrieving Results for Index 470.
Missing field/result... skipping.
------------


In [10]:
# build new dataframe for just cities with airports in the ideal dataframe
narrowed_df = hotel_df.loc[hotel_df['Hotel Name']!='']
narrowed_df

,City,Country,Date (UTC),Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind (mph),Hotel Name
242,nioaque,BR,2020-04-11 19:35:43,-21.14,-55.83,78.30,49.0,0.0,4.41,Nioaque Airport
311,moron,AR,2020-04-11 19:32:12,-34.65,-58.62,75.00,49.0,0.0,6.93,Morón Air Base
395,charagua,BO,2020-04-11 19:38:53,-19.80,-63.22,78.64,29.0,0.0,5.14,Charagua Airport
429,vavuniya,LK,2020-04-11 19:39:56,8.75,80.50,75.11,89.0,0.0,2.82,Vavuniya Airport


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_df.iterrows()]
locations = narrowed_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))